## Construyendo un Agente de Azure AI con Plugins

![Flujo de Trabajo del Agente](https://github.com/kuljotSB/semantic-kernel/blob/main/Assets/AzureAIAgent_With_Plugins.jpg?raw=true)


#### Instalando dependencias y bibliotecas

In [ ]:
%pip install semantic-kernel==1.28.1, azure-identity, python-dotenv, azure-ai-projects==1.0.0b7

#### Cargando nuestras herramientas: OpenAPI y Code Interpreter

In [ ]:
# --- Importaciones Necesarias ---
from azure.identity import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import MessageTextContent
from dotenv import load_dotenv
# Importaciones para la herramienta de API externa (OpenAPI).
from azure.ai.projects.models import OpenApiTool, OpenApiAnonymousAuthDetails
import asyncio
from typing import Any, Callable, Set, Dict, List, Optional
import jsonref
from azure.ai.projects.models import FunctionTool, ToolSet
import json
# Importaciones para la herramienta de Intérprete de Código.
from azure.ai.projects.models import CodeInterpreterTool, MessageAttachment

# Se carga el archivo .env.
load_dotenv()

# Se obtiene el nombre del modelo a utilizar.
model = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_MODEL")

# Se crea el cliente para conectar con el Servicio de Agentes de Azure AI.
project_client = AzureAIAgent.create_client(credential=DefaultAzureCredential(),
                                          conn_str=os.getenv("PROJECT_CONNECTION_STRING")
)

# --- Herramienta 1: OpenAPI para el Clima ---
# Se lee la especificación OpenAPI desde un archivo local.
with open("./weather_openapi.json", "r") as f:
    openapi_spec = jsonref.loads(f.read())

# Se crea un objeto de autenticación (anónima en este caso).
auth = OpenApiAnonymousAuthDetails()

# Se inicializa la herramienta OpenAPI, dándole un nombre, la especificación y la autenticación.
openapi = OpenApiTool(
    name="get_weather", spec=openapi_spec, description="Recupera información del clima para una ubicación", auth=auth
)

# --- Herramienta 2: Intérprete de Código ---
# Se inicializa la herramienta de Intérprete de Código. Es así de simple.
code_interpreter = CodeInterpreterTool()

#### Encapsulando ambas Herramientas en un `ToolSet`

In [ ]:
# Un 'ToolSet' es un contenedor para agrupar múltiples herramientas.
toolset = ToolSet()
# Se añade la herramienta de OpenAPI al conjunto.
toolset.add(openapi)
# Se añade la herramienta de Intérprete de Código al mismo conjunto.
toolset.add(code_interpreter)

#### Creando el Agente de Azure AI (en la Nube)

In [ ]:
# Esta llamada a la API crea y registra un nuevo agente en el servicio de Azure AI.
# Es crucial que le pasemos el 'toolset' para que el agente "nazca" sabiendo usar estas herramientas.
agent = await project_client.agents.create_agent(
        model=model,
        name="multiple-tools-assistant",
        instructions="Eres un asistente muy útil",
        toolset=toolset, # Aquí se le equipa con el conjunto de herramientas.
    )

print(f"Agente creado en la nube, ID: {agent.id}")
# Se guarda el ID del agente para usarlo más adelante.
agent_id = agent.id

# Se crea un hilo de conversación en el servicio de Azure para este agente.
thread = await project_client.agents.create_thread()
print(f"Hilo creado en la nube, ID del hilo: {thread.id}")
# Se guarda el ID del hilo.
thread_id = thread.id

#### Creando el Agente de Azure AI (en Semantic Kernel ~ Localmente)

In [ ]:
# Se obtiene la definición completa del agente que acabamos de crear en la nube.
agent_definition = await project_client.agents.get_agent(agent_id=agent_id)
# Se crea el objeto 'AzureAIAgent' en nuestro código local. Este actuará como un "control remoto"
# para el agente que vive en Azure.
agent = AzureAIAgent(client=project_client, definition=agent_definition)

#### Invocando al Agente con un Hilo de Conversación

In [ ]:
# Se importa la clase para manejar hilos de conversación de este tipo de agente.
from semantic_kernel.agents import AzureAIAgentThread

# Se crea el objeto 'thread' local, vinculándolo al hilo que ya existe en la nube usando su ID.
thread: AzureAIAgentThread = AzureAIAgentThread(client=project_client, thread_id=thread_id)
print(f"Trabajando con el hilo, ID: {thread.id}")

# Se define una pregunta compleja que requiere ambas herramientas.
user_input = "¿Cuál es el clima en París y genera un gráfico para el mismo?"

# Se define la función asíncrona para llamar al agente.
async def get_response_from_agent():
    # Al llamar a 'get_response', el agente en Azure recibirá la pregunta.
    # Razonará que primero debe usar la herramienta de clima y luego la de intérprete de código.
    response =  await agent.get_response(
        messages = user_input,
        thread=thread, # Se pasa el hilo para mantener el historial de la conversación.
    )
    
    return response

# Se ejecuta la llamada y se imprime la respuesta de texto del agente.
response = await get_response_from_agent()
print(response)

#### Recuperando el archivo PNG del Gráfico

In [ ]:
# Importaciones adicionales para manejar archivos y roles de mensajes.
from azure.ai.projects.models import CodeInterpreterTool, MessageAttachment
from azure.ai.projects.models import FilePurpose, MessageRole
from pathlib import Path

# Se recupera el historial completo de mensajes del hilo desde el servicio de Azure.
messages = await project_client.agents.list_messages(thread_id=thread_id)
print(f"Mensajes recuperados: {messages}")

# Se obtiene el último mensaje de texto del agente (si lo hay).
last_msg = messages.get_last_text_message_by_role(MessageRole.AGENT)
if last_msg:
        print(f"Último Mensaje de texto: {last_msg.text.value}")

# Se itera sobre cualquier contenido de imagen que haya en los mensajes.
# Esto encontrará el gráfico generado por el Intérprete de Código.
for image_content in messages.image_contents:
        print(f"ID del archivo de imagen encontrado: {image_content.image_file.file_id}")
        # Se crea un nombre de archivo local único.
        file_name = f"{image_content.image_file.file_id}_image_file.png"
        # Se descarga y guarda el archivo de imagen en el disco local.
        await project_client.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
        print(f"Archivo de imagen guardado en: {Path.cwd() / file_name}")